In [ ]:
from flask import Flask, request, render_template_string
from meals import MEALS

def lbs_to_kg(lbs):
    return lbs * 0.453592

def inches_to_cm(inches):
    return inches * 2.54

app = Flask(__name__)

# -------------------------
# Calorie estimation
# -------------------------
def estimate_daily_calories(weight_kg, height_cm):
    # simplified BMR + activity
    bmr = 10 * weight_kg + 6.25 * height_cm - 100
    activity_factor = 1.6
    return int(bmr * activity_factor)

# -------------------------
# Meal filtering logic
# -------------------------
def recommend_meals(timing, daily_calories):
    """
    Return meals matching timing.
    Later we can score by calorie needs.
    """
    matches = [m for m in MEALS if m["timing"] == timing]

    # simple logic: sort by calories (closest to 25% of daily intake)
    target = daily_calories * 0.25

    matches.sort(key=lambda m: abs(m["calories"] - target))

    return matches[:3]

# -------------------------
# HTML
# -------------------------
HTML = """
<!doctype html>
<title>Pickleball Fuel Finder</title>

<h1>🏓 Pickleball Meal Recommendations</h1>

<form method="post">

  <label>Height:</label><br>
  <input type="number" name="height" required>

  <select name="height_unit">
    <option value="cm">cm</option>
    <option value="in">in</option>
  </select>
  <br><br>

  <select name="weight_unit">
    <option value="kg">kg</option>
    <option value="lb">lb</option>
  </select>
  <br><br>

  <label>How long are you playing?</label><br>
  <select name="duration">
    <option value="short">Less than 1 hour</option>
    <option value="medium">1–3 hours</option>
    <option value="long">Tournament / 3+ hours</option>
  </select><br><br>

  <label>When do you want food for?</label><br>
  <select name="timing">
    <option value="pre">Before playing</option>
    <option value="during">Between games</option>
    <option value="post">After playing</option>
  </select><br><br>

  <input type="submit" value="Get Recommendations">

</form>

{% if daily_cal %}
<hr>
<h3>Estimated Daily Calories: {{ daily_cal }} kcal</h3>
{% endif %}

{% if meals %}
<h2>Top Meal Picks:</h2>
<ul>
{% for meal in meals %}
  <li>
    <b>{{ meal.name }}</b> — {{ meal.calories }} kcal<br>
    Carbs: {{ meal.carbs }}g | Protein: {{ meal.protein }}g | Fat: {{ meal.fat }}g
  </li><br>
{% endfor %}
</ul>
{% endif %}
"""

# -------------------------
# Route
# -------------------------
@app.route("/", methods=["GET", "POST"])
def home():
    meals = None
    daily_cal = None

    if request.method == "POST":
        height = float(request.form.get("height"))
        h_unit = request.form.get("height_unit")
        if h_unit == "in":
            height = inches_to_cm(height)
        weight = float(request.form.get("weight"))
        unit = request.form.get("weight_unit")
        if unit == "lb":
          weight = lbs_to_kg(weight)
        timing = request.form.get("timing")

        daily_cal = estimate_daily_calories(weight, height)
        meals = recommend_meals(timing, daily_cal)

    return render_template_string(HTML, meals=meals, daily_cal=daily_cal)


if __name__ == "__main__":
    app.run(debug=True)
